In [1]:
"""
Import package of dealing with Fermi data
"""
from gw_grb_correlation.Fermi.data_preprocessing import create_dataframe_and_name_column_from_data_files, download_and_preprocess_fermi_data
from gw_grb_correlation.Fermi.download_data_functions import download_file


In [ ]:
"""
Download and preprocess Fermi data
"""
start_year = 2015
end_year = 2026
fermi_data = download_and_preprocess_fermi_data(start_year=start_year, end_year=end_year, download_or_not=True)

In [ ]:
"""
Optional: If you don't want to download the raw data from Fermi database (which will take 12 hours), you can download the preporcessed data from the following link
Note that this data only conatins the Fermi data from 2015 to 2025 April. 
"""
import os

os.makedirs("fermi_data", exist_ok=True)
os.makedirs("fermi_data/fermi", exist_ok=True)

"""
Google Drive download URLs
"""
downloads = {
    "https://drive.google.com/uc?id=18fEbzKDVn8_-Y8d8MnQLKrAgSFT4Vmlu": "./fermi_data/fermi/fermi_data.npy",
}

"""
Download files
"""
for url, output_path in downloads.items():
    download_file(url, output_path)

fermi_data = create_dataframe_and_name_column_from_data_files(data_type="fermi")

In [ ]:
"""
Extract time data, and plot histograms of time and T90 duration of GRBs
"""
from gw_grb_correlation.Fermi.visualization import create_time_data_plots

create_time_data_plots(fermi_data, 'plots')

In [ ]:
"""
Extract location data, and plot histograms of location data of GRBs
"""
from gw_grb_correlation.Fermi.visualization import create_location_data_plots
create_location_data_plots(fermi_data, 'plots')

In [ ]:
"""
Plot the poseterior of GRB location of a specific GRB event
"""
from gw_grb_correlation.Fermi.visualization import plot_certain_event_prob_dist

url_locprob = "https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/2017/bn170817529/current/glg_locprob_all_bn170817529_v02.fit"
filename_locprob = "glg_locprob_all_bn170817529_v02.fit"

download_file(url_locprob, filename_locprob)
plot_certain_event_prob_dist(filename_locprob, 'plots')

In [ ]:
"""
Plot the pointing of all detectors on Fermi GBM on the skymap during one GRB event
"""
from gw_grb_correlation.Fermi.visualization import plot_all_detector_positions
plot_all_detector_positions(fermi_data.head(1), plt_show_or_not=True)

In [ ]:
from gw_grb_correlation.Fermi.visualization import plot_light_curve_with_baseline_subtraction
"""
Download the tte data for the specific event
"""
url_tte = "https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/2017/bn170817529/current/glg_tte_n2_bn170817529_v00.fit"
filename_tte = "glg_tte_n2_bn170817529_v00.fit"

download_file(url_tte, filename_tte)

"""
Download the bcat data for the specific event
"""
url_bcat = "https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/2017/bn170817529/current/glg_bcat_all_bn170817529_v01.fit"
filename_bcat = "glg_bcat_all_bn170817529_v01.fit"

download_file(url_bcat, filename_bcat)

"""
Plot the light curve with baseline subtraction
"""
plot_light_curve_with_baseline_subtraction(filename_tte, filename_bcat, output_folder='./plots')

In [ ]:
from gw_grb_correlation.Fermi.util import duration, filtering, read_GW_data
"""
Filter out short GRB data
"""
short_GRB_data = filtering(fermi_data, criteria={'T90': lambda x: x <= 2.1})

"""
Calculate the duration (difference between max TSTOP and min TSTART) and count the short GRB event number
"""
duration_short_GRB = duration(short_GRB_data)
event_num_short_GRB = len(short_GRB_data)

"""
Calculate the average occurrence rate of short GRB (events per unit of time)
"""
average_occurrence_rate = event_num_short_GRB / duration_short_GRB

"""
Print the results
"""
print(f"Number of short GRB: {event_num_short_GRB}")
print(f"Total duration: {duration_short_GRB} seconds")
print(f"Average occurrence rate: {average_occurrence_rate} short GRB per second")

In [ ]:
from gw_grb_correlation.Fermi.util import read_GW_data, remove_duplicate_times_in_gw_data, compare_time_within_range
"""
Load GW data
To run the time correlation analysis between GRB and GW, be sure that you already run the GW data preprocessing notebook first to get the totalgwdata.csv file and places it in the ./gw_data/ directory.
"""
gw_data = read_GW_data(f"./gw_data/totalgwdata.csv")
gw_times = remove_duplicate_times_in_gw_data(gw_data)

"""
Find matched GRB-GW event pairs
"""
match = compare_time_within_range(short_GRB_data, gw_times, time_range_seconds=86400*3)
filtered_gw_events = gw_data[gw_data['times'].isin(match['gw_time'])]

"""
Save the matched GRB-GW event pairs and filtered GW events to CSV files
"""
match.to_csv("GRB_GW_event_pairs.csv", index=False)
filtered_gw_events.to_csv("Filtered_GW_events.csv", index=False)